In [18]:
#SS_CRPS-->CRPS_SS

In [26]:
from datetime import timedelta, date, datetime
import os
import sys
import numpy as np
import csv

climatology_num=1
interval_band=[90,75,50,25,10]

class aaa(object):
    def __init__(self,lead,year_generate=1997):
        self.ensemble_access=['e01','e02','e03','e04','e05','e06','e07','e08','e09','e10','e11']
        self.lead_time=lead
        self.year_generate=year_generate
        self.files=self.get_filename_with_time_order()
    def get_filename_with_time_order(self):
        _files = []
        for mm in range(1,13):
            for dd in [1,9,17,25]:
#                 for i in range(self.lead_time,self.lead_time+1):
#                 for en in ['e01','e02','e03','e04','e05','e06','e07','e08','e09','e10','e11']:
                path=[]
                date_time=date(self.year_generate, mm, dd)
                barra_date=date_time+timedelta(self.lead_time)
                path.append(date_time)
                path.append(barra_date)
                path.append(self.lead_time)
                _files.append(path)
        return _files
    def __getitem__(self,idx):
        return self.files[idx]

def load_climatology_data(l,year_generate):
    data=aaa(l,year_generate)
    climtology_lead_time=[]
    climatology_data=np.load('./save/crps/climatology_'+str(year_generate)+'_all_lead_time_windows_'+str(climatology_num)+'.npy')
#     print(climatology_data.shape)
    dates_needs=date_range(date(year_generate, 1, 1),date(year_generate+1, 7, 29))
    date_map=np.array(dates_needs)
    for _,target_date,_ in data.files:
        idx=np.where(date_map==target_date)[0]
        climtology_lead_time.append(climatology_data[idx][0])
    return np.array(climtology_lead_time)

def date_range(start_date, end_date):
    """This function takes a start date and an end date as datetime date objects.
    It returns a list of dates for each date in order starting at the first date and ending with the last date"""
    return [start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]



import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages



def table_csv_confidence(data_np,file_dir):
    with open(file_dir+'.csv', "w", newline='') as file:
        csv_file = csv.writer(file)
        head = ["leading",str(interval_band[0]),str(interval_band[1]),str(interval_band[2]),str(interval_band[3]),str(interval_band[4])]
        csv_file.writerow(head)

        for lead_time in range(217):
            line=[lead_time,data_np[0,lead_time],data_np[1,lead_time],data_np[2,lead_time],data_np[3,lead_time],data_np[4,lead_time]]

            csv_file.writerow(line)

def draw_plot(data,name,ylim_set=(-0.6,0.5)):
        x=list(range(1,218))
        data_np=np.array(data)
        # plt.plot(x,mean_my,label="",color="b")#mean
        plt.figure(dpi=100,figsize=(15,5))
        plt.plot(x,data_np[2,:],label="",color="r")#median
        plt.plot(x,[0]*217,color="#000000")
        plt.fill_between(x,data_np[0,:],data_np[1,:],color="#cacaca")
        plt.fill_between(x,data_np[1,:],data_np[2,:],color="#989898")
        plt.fill_between(x,data_np[2,:],data_np[3,:],color="#989898")
        plt.fill_between(x,data_np[3,:],data_np[4,:],color="#cacaca")
        # plt.xlim(0,217)
        # plt.grid()
        plt.xlabel(" Leadtime (day)")
        plt.ylabel(" CRPS_SS")
        plt.ylim(ylim_set[0],ylim_set[1])
        pdf = PdfPages(name+".pdf")
        pdf.savefig()
        pdf.close()
        plt.clf()
        table_csv_confidence(data_np,name)
            


            
            
def save_csv_mean(my_pr=0,my_pr_zg=0,cali=0,BI=0,climat=0,ss_my_pr=0,ss_my_pr_zg=0,ss_cali=0,ss_BI=0,lead=0,file_dir=0,first_init=0):
    if first_init:
        with open('table_mean_'+file_dir+'.csv', "w", newline='') as file:
            csv_file = csv.writer(file)
            head = ["leading","climatology_"+str(climatology_num),'bilinear interpolation','vdsrd','vdsrd2','calibration','cspr_ss_mean_bilinearinterpolation_against_climatology','cspr_ss_mean_vdsrd_against_climatology','cspr_ss_mean_vdsrd2_against_climatology','cspr_ss_mean_calibration_against_climatology']
            csv_file.writerow(head)
    else:
        line=[lead,climat.mean(),BI.mean(),my_pr.mean(),my_pr_zg.mean(),cali.mean(),ss_BI.mean(), ss_my_pr.mean(),ss_my_pr_zg.mean(),ss_cali.mean()]
        with open('table_mean_'+file_dir+'.csv', "a", newline='') as file:
            csv_file = csv.writer(file)
#             head = ["leading","climatology_"+str(climatology_num),'bilinear interpolation','vdsrd','vdsrd2','calibration','cspr_ss_mean_bilinearinterpolation_against_climatology','cspr_ss_mean_vdsrd_against_climatology','cspr_ss_mean_vdsrd2_against_climatology','cspr_ss_mean_calibration_against_climatology']
            csv_file.writerow(line) 
    
def save_csv_median(my_pr=0,my_pr_zg=0,cali=0,BI=0,climat=0,ss_my_pr=0,ss_my_pr_zg=0,ss_cali=0,ss_BI=0,lead=0,file_dir=0,first_init=0):
    if first_init:
        with open('table_median_'+file_dir+'.csv', "w", newline='') as file:
            csv_file = csv.writer(file)
            head = ["leading","climatology_"+str(climatology_num),'bilinear interpolation','vdsrd','vdsrd2','calibration','cspr_ss_mean_bilinearinterpolation_against_climatology','cspr_ss_mean_vdsrd_against_climatology','cspr_ss_mean_vdsrd2_against_climatology','cspr_ss_mean_calibration_against_climatology']
            csv_file.writerow(head)
    else:
        line=[lead,np.median(climat),np.median(BI),np.median(my_pr),np.median(my_pr_zg),np.median(cali),np.median(ss_BI), np.median(ss_my_pr),np.median(ss_my_pr_zg),np.median(ss_cali)]
        with open('table_median_'+file_dir+'.csv', "a", newline='') as file:
            csv_file = csv.writer(file)
#             head = ["leading","climatology_"+str(climatology_num),'bilinear interpolation','vdsrd','vdsrd2','calibration','cspr_ss_mean_bilinearinterpolation_against_climatology','cspr_ss_mean_vdsrd_against_climatology','cspr_ss_mean_vdsrd2_against_climatology','cspr_ss_mean_calibration_against_climatology']
            csv_file.writerow(line)           
            
            
            
def draw_confidence_figure(option='whole',year_generate=2010):
    
    if option == "whole":
        file_path='whole_climatology'+str(climatology_num)
        
        
        save_csv_mean(file_dir=file_path,first_init=1)
        save_csv_mean(file_dir=file_path,first_init=1)

        land=np.load("./save/crps/whole_calibration/2012/lead_time"+str(0)+"_whole.npy").mean(0)
        x=list(range(1,218))



        data_my_pr_against_climatology=[]
        data_my_pr_zg_against_climatology=[]
        data_cali_against_climatology=[]
        data_BI_against_climatology=[]

        
        mean_my=[]

        for q in interval_band:
            my_pr_against_climatology=[]
            my_pr_zg_against_climatology=[]
            cali_against_climatology=[]
            BI_against_climatology=[]
            
            for lead_time in range(217):
                if year_generate==2012:
                    my_pr=np.load("./save/crps/val"+str(year_generate)[-2:]+"/lead_time"+str(lead_time)+".npy").mean(0)[~np.isinf(land)]
                    my_pr_zg=np.load("/scratch/iu60/mc7437/zg/save/crps/wjdata/lead_time"+str(lead_time)+".npy").mean(0)[~np.isinf(land)]

                    BI=np.load("./save/crps/bi_217/"+str(year_generate)+"/lead_time"+str(lead_time)+".npy").mean(0)[~np.isinf(land)]
                    climat=load_climatology_data(lead_time,year_generate).mean(0)[~np.isinf(land)]
                    cali=np.load("./save/crps/whole_calibration/"+str(year_generate)+"/lead_time"+str(lead_time)+"_whole.npy").mean(0)[~np.isinf(land)]

                else:
                    my_pr=np.load("/scratch/iu60/mc7437/baseline/save/crps/val"+str(year_generate)[-2:]+"/lead_time"+str(lead_time)+".npy").mean(0)[~np.isinf(land)]
                    my_pr_zg=np.load("/scratch/iu60/mc7437/zg/save/crps/val"+str(year_generate)[-2:]+"/lead_time"+str(lead_time)+".npy").mean(0)[~np.isinf(land)]
                    climat=load_climatology_data(lead_time,year_generate).mean(0)[~np.isinf(land)]
                    cali=np.load("./save/crps/whole_calibration/"+str(year_generate)+"/lead_time"+str(lead_time)+"_whole.npy").mean(0)[~np.isinf(land)]
                    BI=np.load("./save/crps/bi_217/"+str(year_generate)+"/lead_time_"+str(lead_time)+".npy").mean(0)[~np.isinf(land)]
                
                ss_my_pr= 1-my_pr/climat
                ss_my_pr_zg=1-my_pr_zg/climat
                ss_cali=1-cali/climat
                ss_bi=1-BI/climat
                
                if q==interval_band[0]: #save four module crps and ss_crps
                    save_csv_mean(my_pr,my_pr_zg,cali, BI,climat, ss_my_pr,ss_my_pr_zg,ss_cali,ss_bi, lead=lead_time,file_dir=file_path )
                    save_csv_median(my_pr,my_pr_zg,cali, BI,climat, ss_my_pr,ss_my_pr_zg,ss_cali,ss_bi, lead=lead_time,file_dir=file_path)

                my_pr_against_climatology.append(np.percentile( ss_my_pr,q))
                my_pr_zg_against_climatology.append(np.percentile(ss_my_pr_zg ,q))
                cali_against_climatology.append(np.percentile( ss_cali,q))
                BI_against_climatology.append(np.percentile(ss_bi ,q))

            data_my_pr_against_climatology.append(my_pr_against_climatology)
            data_my_pr_zg_against_climatology.append(my_pr_zg_against_climatology)
            data_cali_against_climatology.append(cali_against_climatology)
            data_BI_against_climatology.append(BI_against_climatology)


        draw_plot(data_my_pr_against_climatology,'DL_pr_only_against_climatology_'+str(year_generate)+'_'+option) 
        draw_plot(data_my_pr_zg_against_climatology,'DL_pr_zg_against_climatology_'+str(year_generate)+'_'+option) 
        draw_plot(data_cali_against_climatology,'calibration_against_climatology_'+str(year_generate)+'_'+option) 
        draw_plot(data_BI_against_climatology,'BI_against_climatology_'+str(year_generate)+'_'+option) 

        
        
        
    else:
        
        file_path='station_climatology'+str(climatology_num)
        save_csv_mean(file_dir=file_path,first_init=1)
        save_csv_mean(file_dir=file_path,first_init=1)
        
        if option=='50':
            from constant_param import station_50_index_for_size_of_hr_sr as station_dict
        if option=='214':
            from constant_param import station_214_index_for_size_of_hr_sr as station_dict
            
        land=np.load("./save/crps/whole_calibration/2012/lead_time"+str(0)+"_whole.npy").mean(0)
        station_index=np.zeros((land.shape))
        for i in station_dict.keys():
            station_index[station_dict[i]]=1
            
        x=list(range(1,218))



        data_my_pr_against_climatology=[]
        data_my_pr_zg_against_climatology=[]
        data_cali_against_climatology=[]
        data_BI_against_climatology=[]

        
        mean_my=[]
        # for i in range(30):
        #     a=np.load("./save_vdsr_pr_best_test/lead_time"+str(i)+"_50station_my.npy")
        #     mean_my.append(1-a.mean()/station50_int[i])
        # for q in [95,75,50,25,5]:
        #     data[q]=[]
        for q in interval_band:
            my_pr_against_climatology=[]
            my_pr_zg_against_climatology=[]
            cali_against_climatology=[]
            BI_against_climatology=[]
            
            for lead_time in range(217):
                if year_generate==2012:
                    my_pr=np.load("./save/crps/val"+str(year_generate)[-2:]+"/lead_time"+str(lead_time)+".npy").mean(0)[~np.isinf(land)]
                    #!!!!!!!!my_pr_zg=np.load("/scratch/iu60/mc7437/zg/save/crps/val"+str(year_generate)[-2:]+"/lead_time"+str(lead_time)+".npy").mean(0)[~np.isinf(land)]
                    my_pr_zg=np.load("/scratch/iu60/mc7437/zg/save/crps/wjdata/lead_time"+str(lead_time)+".npy").mean(0)[~np.isinf(land)]

                    BI=np.load("./save/crps/bi_217/"+str(year_generate)+"/lead_time"+str(lead_time)+".npy").mean(0)[~np.isinf(land)]
                    climat=load_climatology_data(lead_time,year_generate).mean(0)[~np.isinf(land)]
                    cali=np.load("./save/crps/whole_calibration/"+str(year_generate)+"/lead_time"+str(lead_time)+"_whole.npy").mean(0)[~np.isinf(land)]

                else:
                    my_pr=np.load("/scratch/iu60/mc7437/baseline/save/crps/val"+str(year_generate)[-2:]+"/lead_time"+str(lead_time)+".npy").mean(0)[~np.isinf(land)]
                    my_pr_zg=np.load("/scratch/iu60/mc7437/zg/save/crps/val"+str(year_generate)[-2:]+"/lead_time"+str(lead_time)+".npy").mean(0)[~np.isinf(land)]
                    climat=load_climatology_data(lead_time,year_generate).mean(0)[~np.isinf(land)]
                    cali=np.load("./save/crps/whole_calibration/"+str(year_generate)+"/lead_time"+str(lead_time)+"_whole.npy").mean(0)[~np.isinf(land)]
                    BI=np.load("./save/crps/bi_217/"+str(year_generate)+"/lead_time_"+str(lead_time)+".npy").mean(0)[~np.isinf(land)]
                

                
                ss_my_pr= 1-my_pr/climat
                ss_my_pr_zg=1-my_pr_zg/climat
                ss_cali=1-cali/climat
                ss_bi=1-BI/climat
                
                if q==interval_band[0]: #save four module crps and ss_crps
                    save_csv_mean(my_pr,my_pr_zg,cali, BI,climat, ss_my_pr,ss_my_pr_zg,ss_cali,ss_bi, lead=lead_time,file_dir=file_path )
                    save_csv_median(my_pr,my_pr_zg,cali, BI,climat, ss_my_pr,ss_my_pr_zg,ss_cali,ss_bi, lead=lead_time,file_dir=file_path )

                my_pr_against_climatology.append(np.percentile( 1-my_pr/climat,q))
                my_pr_zg_against_climatology.append(np.percentile( 1-my_pr_zg/climat,q))
                cali_against_climatology.append(np.percentile( 1-cali/climat,q))
                BI_against_climatology.append(np.percentile( 1-BI/climat,q))

            data_my_pr_against_climatology.append(my_pr_against_climatology)
            data_my_pr_zg_against_climatology.append(my_pr_zg_against_climatology)
            data_cali_against_climatology.append(cali_against_climatology)
            data_BI_against_climatology.append(BI_against_climatology)

        draw_plot(data_my_pr_against_climatology,'DL_pr_only_against_climatology_'+str(year_generate)+'_'+option) 
        draw_plot(data_my_pr_zg_against_climatology,'DL_pr_zg_against_climatology_'+str(year_generate)+'_'+option) 
        draw_plot(data_cali_against_climatology,'calibration_against_climatology_'+str(year_generate)+'_'+option) 
        draw_plot(data_BI_against_climatology,'BI_against_climatology_'+str(year_generate)+'_'+option) 
        

        

                                     
draw_confidence_figure(option='whole',year_generate=2012)
draw_confidence_figure(option='50',year_generate=2012)
draw_confidence_figure(option='whole',year_generate=2010)
draw_confidence_figure(option='50',year_generate=2010)
draw_confidence_figure(option='whole',year_generate=1997)
draw_confidence_figure(option='50',year_generate=1997)

In [35]:
# np.median(np.array([0,1,23,21,32,5,654,5]))
np.percentile( np.array([0,1,23,21,32,5,654,5]),50)

13.0

In [ ]:
from datetime import timedelta, date, datetime
import os
import sys
import numpy as np

class aaa(object):
    def __init__(self,lead,year_generate=1997):
        self.ensemble_access=['e01','e02','e03','e04','e05','e06','e07','e08','e09','e10','e11']
        self.lead_time=lead
        self.year_generate=year_generate
        self.files=self.get_filename_with_time_order()
    def get_filename_with_time_order(self):
        _files = []
        for mm in range(1,13):
            for dd in [1,9,17,25]:
#                 for i in range(self.lead_time,self.lead_time+1):
#                 for en in ['e01','e02','e03','e04','e05','e06','e07','e08','e09','e10','e11']:
                path=[]
                date_time=date(self.year_generate, mm, dd)
                barra_date=date_time+timedelta(self.lead_time)
                path.append(date_time)
                path.append(barra_date)
                path.append(self.lead_time)
                _files.append(path)
        return _files
    def __getitem__(self,idx):
        return self.files[idx]

def load_climatology_data(l,year_generate):
    data=aaa(l,year_generate)
    climtology_lead_time=[]
    climatology_data=np.load('./save/crps/climatology_'+str(year_generate)+'_all_lead_time_windows_1.npy')
#     print(climatology_data.shape)
    dates_needs=date_range(date(year_generate, 1, 1),date(year_generate+1, 7, 29))
    date_map=np.array(dates_needs)
    for _,target_date,_ in data.files:
        idx=np.where(date_map==target_date)[0]
        climtology_lead_time.append(climatology_data[idx][0])
    return np.array(climtology_lead_time)

def date_range(start_date, end_date):
    """This function takes a start date and an end date as datetime date objects.
    It returns a list of dates for each date in order starting at the first date and ending with the last date"""
    return [start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]



import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages


def draw_confidence_figure(option='whole',year_generate=2010):
    if option == "whole":

        land=np.load("./save/crps/whole_calibration/2012/lead_time"+str(0)+"_whole.npy").mean(0)
        x=list(range(1,218))



        data_my_pr_against_climatology=[]
        data_my_pr_zg_against_climatology=[]
        data_cali_against_climatology=[]
        data_BI_against_climatology=[]

        
        mean_my=[]
        # for i in range(30):
        #     a=np.load("./save_vdsr_pr_best_test/lead_time"+str(i)+"_50station_my.npy")
        #     mean_my.append(1-a.mean()/station50_int[i])
        # for q in [95,75,50,25,5]:
        #     data[q]=[]
        for q in [90,75,50,25,10]:
            my_pr_against_climatology=[]
            my_pr_zg_against_climatology=[]
            cali_against_climatology=[]
            BI_against_climatology=[]
            
            for lead_time in range(217):
                my_pr=np.load("/scratch/iu60/mc7437/baseline/save/crps/val"+str(year_generate)[-2:]+"/lead_time"+str(lead_time)+".npy").mean(0)[~np.isinf(land)]
                my_pr_zg=np.load("/scratch/iu60/mc7437/zg/save/crps/val"+str(year_generate)[-2:]+"/lead_time"+str(lead_time)+".npy").mean(0)[~np.isinf(land)]
                climat=load_climatology_data(lead_time,year_generate).mean(0)[~np.isinf(land)]
                cali=np.load("./save/crps/whole_calibration/"+str(year_generate)+"/lead_time"+str(lead_time)+"_whole.npy").mean(0)[~np.isinf(land)]
                BI=np.load("./save/crps/bi_217/"+str(year_generate)+"/lead_time_"+str(lead_time)+".npy").mean(0)[~np.isinf(land)]
                
                my_pr_against_climatology.append(np.percentile( 1-my_pr/climat,q))
                my_pr_zg_against_climatology.append(np.percentile( 1-my_pr_zg/climat,q))
                cali_against_climatology.append(np.percentile( 1-cali/climat,q))
                BI_against_climatology.append(np.percentile( 1-BI/climat,q))

            data_my_pr_against_climatology.append(my_pr_against_climatology)
            data_my_pr_zg_against_climatology.append(my_pr_zg_against_climatology)
            data_cali_against_climatology.append(cali_against_climatology)
            data_BI_against_climatology.append(BI_against_climatology)

        data_np=np.array(data_my_pr_against_climatology)
        # plt.plot(x,mean_my,label="",color="b")#mean
        plt.figure(dpi=100,figsize=(15,5))
        plt.plot(x,data_np[2,:],label="",color="r")#median
        plt.plot(x,[0]*217,color="#000000")
        plt.fill_between(x,data_np[0,:],data_np[1,:],color="#cacaca")
        plt.fill_between(x,data_np[1,:],data_np[2,:],color="#989898")
        plt.fill_between(x,data_np[2,:],data_np[3,:],color="#989898")
        plt.fill_between(x,data_np[3,:],data_np[4,:],color="#cacaca")
        # plt.xlim(0,217)
        # plt.grid()
        plt.xlabel(" Leadtime (day)")
        plt.ylabel(" SS_CRPS")
        plt.ylim(-0.6,0.5)
        pdf = PdfPages('DL_pr_only_against_climatology_'+str(year_generate)+'_'+option+".pdf")
        pdf.savefig()
        pdf.close()
        plt.clf()
        
        data_np=np.array(data_my_pr_zg_against_climatology)
        # plt.plot(x,mean_my,label="",color="b")#mean
        plt.figure(dpi=100,figsize=(15,5))
        plt.plot(x,data_np[2,:],label="",color="r")#median
        plt.plot(x,[0]*217,color="#000000")
        plt.fill_between(x,data_np[0,:],data_np[1,:],color="#cacaca")
        plt.fill_between(x,data_np[1,:],data_np[2,:],color="#989898")
        plt.fill_between(x,data_np[2,:],data_np[3,:],color="#989898")
        plt.fill_between(x,data_np[3,:],data_np[4,:],color="#cacaca")
        # plt.xlim(0,217)
        # plt.grid()
        plt.xlabel(" Leadtime (day)")
        plt.ylabel(" SS_CRPS")
        plt.ylim(-0.6,0.5)
        pdf = PdfPages('DL_pr_zg_against_climatology_'+str(year_generate)+'_'+option+".pdf")
        pdf.savefig()
        pdf.close()
        plt.clf()
        
        
        data_np=np.array(data_cali_against_climatology)
        # plt.plot(x,mean_my,label="",color="b")#mean
        plt.figure(dpi=100,figsize=(15,5))
        plt.plot(x,data_np[2,:],label="",color="r")#median
        plt.plot(x,[0]*217,color="#000000")
        plt.fill_between(x,data_np[0,:],data_np[1,:],color="#cacaca")
        plt.fill_between(x,data_np[1,:],data_np[2,:],color="#989898")
        plt.fill_between(x,data_np[2,:],data_np[3,:],color="#989898")
        plt.fill_between(x,data_np[3,:],data_np[4,:],color="#cacaca")
        # plt.xlim(0,217)
        # plt.grid()
        plt.xlabel(" Leadtime (day)")
        plt.ylabel(" SS_CRPS")
        plt.ylim(-0.6,0.5)
        pdf = PdfPages('calibration_against_climatology_'+str(year_generate)+'_'+option+".pdf")
        pdf.savefig()
        pdf.close()
        plt.clf()  
        
        data_np=np.array(data_BI_against_climatology)
        # plt.plot(x,mean_my,label="",color="b")#mean
        plt.figure(dpi=100,figsize=(15,5))
        plt.plot(x,data_np[2,:],label="",color="r")#median
        plt.plot(x,[0]*217,color="#000000")
        plt.fill_between(x,data_np[0,:],data_np[1,:],color="#cacaca")
        plt.fill_between(x,data_np[1,:],data_np[2,:],color="#989898")
        plt.fill_between(x,data_np[2,:],data_np[3,:],color="#989898")
        plt.fill_between(x,data_np[3,:],data_np[4,:],color="#cacaca")
        # plt.xlim(0,217)
        # plt.grid()
        plt.xlabel(" Leadtime (day)")
        plt.ylabel(" SS_CRPS")
        plt.ylim(-0.6,0.5)
        pdf = PdfPages('BI_against_climatology_'+str(year_generate)+'_'+option+".pdf")
        pdf.savefig()
        pdf.close()
        plt.clf()        
        
    else:
        if option=='50':
            from constant_param import station_50_index_for_size_of_hr_sr as station_dict
        if option=='214':
            from constant_param import station_214_index_for_size_of_hr_sr as station_dict
            
        land=np.load("./save/crps/whole_calibration/2012/lead_time"+str(0)+"_whole.npy").mean(0)
        station_index=np.zeros((land.shape))
        for i in station_dict.keys():
            station_index[station_dict[i]]=1
            
        x=list(range(1,218))



        data_my_pr_against_climatology=[]
        data_my_pr_zg_against_climatology=[]
        data_cali_against_climatology=[]
        data_BI_against_climatology=[]

        
        mean_my=[]
        # for i in range(30):
        #     a=np.load("./save_vdsr_pr_best_test/lead_time"+str(i)+"_50station_my.npy")
        #     mean_my.append(1-a.mean()/station50_int[i])
        # for q in [95,75,50,25,5]:
        #     data[q]=[]
        for q in [90,75,50,25,10]:
            my_pr_against_climatology=[]
            my_pr_zg_against_climatology=[]
            cali_against_climatology=[]
            BI_against_climatology=[]
            
            for lead_time in range(217):
                my_pr=np.load("/scratch/iu60/mc7437/baseline/save/crps/val"+str(year_generate)[-2:]+"/lead_time"+str(lead_time)+".npy").mean(0)[station_index==1]
                my_pr_zg=np.load("/scratch/iu60/mc7437/zg/save/crps/val"+str(year_generate)[-2:]+"/lead_time"+str(lead_time)+".npy").mean(0)[station_index==1]
                climat=load_climatology_data(lead_time,year_generate).mean(0)[station_index==1]
                cali=np.load("./save/crps/whole_calibration/"+str(year_generate)+"/lead_time"+str(lead_time)+"_whole.npy").mean(0)[station_index==1]
                BI=np.load("./save/crps/bi_217/"+str(year_generate)+"/lead_time_"+str(lead_time)+".npy").mean(0)[station_index==1]
                
                my_pr_against_climatology.append(np.percentile( 1-my_pr/climat,q))
                my_pr_zg_against_climatology.append(np.percentile( 1-my_pr_zg/climat,q))
                cali_against_climatology.append(np.percentile( 1-cali/climat,q))
                BI_against_climatology.append(np.percentile( 1-BI/climat,q))

            data_my_pr_against_climatology.append(my_pr_against_climatology)
            data_my_pr_zg_against_climatology.append(my_pr_zg_against_climatology)
            data_cali_against_climatology.append(cali_against_climatology)
            data_BI_against_climatology.append(BI_against_climatology)

        data_np=np.array(data_my_pr_against_climatology)
        # plt.plot(x,mean_my,label="",color="b")#mean
        plt.figure(dpi=100,figsize=(15,5))
        plt.plot(x,data_np[2,:],label="",color="r")#median
        plt.plot(x,[0]*217,color="#000000")
        plt.fill_between(x,data_np[0,:],data_np[1,:],color="#cacaca")
        plt.fill_between(x,data_np[1,:],data_np[2,:],color="#989898")
        plt.fill_between(x,data_np[2,:],data_np[3,:],color="#989898")
        plt.fill_between(x,data_np[3,:],data_np[4,:],color="#cacaca")
        # plt.xlim(0,217)
        # plt.grid()
        plt.xlabel(" Leadtime (day)")
        plt.ylabel(" SS_CRPS")
        plt.ylim(-0.6,0.5)
        pdf = PdfPages('DL_pr_only_against_climatology_'+str(year_generate)+'_'+option+".pdf")
        pdf.savefig()
        pdf.close()
        plt.clf()
        
        data_np=np.array(data_my_pr_zg_against_climatology)
        # plt.plot(x,mean_my,label="",color="b")#mean
        plt.figure(dpi=100,figsize=(15,5))
        plt.plot(x,data_np[2,:],label="",color="r")#median
        plt.plot(x,[0]*217,color="#000000")
        plt.fill_between(x,data_np[0,:],data_np[1,:],color="#cacaca")
        plt.fill_between(x,data_np[1,:],data_np[2,:],color="#989898")
        plt.fill_between(x,data_np[2,:],data_np[3,:],color="#989898")
        plt.fill_between(x,data_np[3,:],data_np[4,:],color="#cacaca")
        # plt.xlim(0,217)
        # plt.grid()
        plt.xlabel(" Leadtime (day)")
        plt.ylabel(" SS_CRPS")
        plt.ylim(-0.6,0.5)
        pdf = PdfPages('DL_pr_zg_against_climatology_'+str(year_generate)+'_'+option+".pdf")
        pdf.savefig()
        pdf.close()
        plt.clf()
        
        
        data_np=np.array(data_cali_against_climatology)
        # plt.plot(x,mean_my,label="",color="b")#mean
        plt.figure(dpi=100,figsize=(15,5))
        plt.plot(x,data_np[2,:],label="",color="r")#median
        plt.plot(x,[0]*217,color="#000000")
        plt.fill_between(x,data_np[0,:],data_np[1,:],color="#cacaca")
        plt.fill_between(x,data_np[1,:],data_np[2,:],color="#989898")
        plt.fill_between(x,data_np[2,:],data_np[3,:],color="#989898")
        plt.fill_between(x,data_np[3,:],data_np[4,:],color="#cacaca")
        # plt.xlim(0,217)
        # plt.grid()
        plt.xlabel(" Leadtime (day)")
        plt.ylabel(" SS_CRPS")
        plt.ylim(-0.6,0.5)
        pdf = PdfPages('calibration_against_climatology_'+str(year_generate)+'_'+option+".pdf")
        pdf.savefig()
        pdf.close()
        plt.clf()  
        
        data_np=np.array(data_BI_against_climatology)
        # plt.plot(x,mean_my,label="",color="b")#mean
        plt.figure(dpi=100,figsize=(15,5))
        plt.plot(x,data_np[2,:],label="",color="r")#median
        plt.plot(x,[0]*217,color="#000000")
        plt.fill_between(x,data_np[0,:],data_np[1,:],color="#cacaca")
        plt.fill_between(x,data_np[1,:],data_np[2,:],color="#989898")
        plt.fill_between(x,data_np[2,:],data_np[3,:],color="#989898")
        plt.fill_between(x,data_np[3,:],data_np[4,:],color="#cacaca")
        # plt.xlim(0,217)
        # plt.grid()
        plt.xlabel(" Leadtime (day)")
        plt.ylabel(" SS_CRPS")
        plt.ylim(-0.6,0.5)
        pdf = PdfPages('BI_against_climatology_'+str(year_generate)+'_'+option+".pdf")
        pdf.savefig()
        pdf.close()
        plt.clf()        
        
        

draw_confidence_figure(option='whole',year_generate=2010)
draw_confidence_figure(option='50',year_generate=2010)
draw_confidence_figure(option='whole',year_generate=1997)
draw_confidence_figure(option='50',year_generate=1997)